In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [ ]:
### This notebook will give you an idea of what my project does. Some parts of this notebook are 
### done differently in my code module, and I have included notes to indicate the differences.

In [ ]:
### Load the skater data for 2011.

In [2]:
df = pd.read_csv("season2011.csv",header=0, encoding='utf_16')

In [ ]:
### Create some variables for processing the data.  
### The first three are for extending player stats to 82 games (full season)
###  in order to provide a way to measure one player against another. 
### 'M'is for handling data on players who were traded during the season.
### 'PEval' is a variable for storing a player value. This variable is named differently in my module.
### 'Status' is for handling the removal of first and second year players
###  since they are exempt from the expansion draft.


In [3]:
df['adjPTS'] = 82*df['PTS']/df['GP']
df['adj+/-'] = 82*df['+/-']/df['GP']
df['adjTOI'] = 82*df['TOI']/df['GP']
df['M'] = 0
df['PEval'] = 0.00
df['Status'] = 0

In [4]:
df.iloc[20:25,:]

,Rk,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,...,HIT,FOW,FOL,FO%,adjPTS,adj+/-,adjTOI,M,PEval,Status
20,21,Keith Aulie\aulieke01,22,TOT,D,36,0,3,3,-7,...,78,0.0,0.0,NaN,6.833333,-15.944444,1102.444444,0,0.0,0
21,21,Keith Aulie\aulieke01,22,TBL,D,19,0,1,1,-5,...,37,0.0,0.0,NaN,4.315789,-21.578947,906.315789,0,0.0,0
22,21,Keith Aulie\aulieke01,22,TOR,D,17,0,2,2,-2,...,41,0.0,0.0,NaN,9.647059,-9.647059,1321.647059,0,0.0,0
23,22,Sean Avery\averyse01,31,NYR,C,15,3,0,3,2,...,10,0.0,3.0,0.0,16.400000,10.933333,574.000000,0,0.0,0
24,23,Anton Babchuk\babchan01,27,CGY,D,32,2,8,10,2,...,17,0.0,0.0,NaN,25.625000,5.125000,1189.000000,0,0.0,0


In [ ]:
### The piece of the dataframe above shows what a trade looks like. Keith Aulie was traded from 
### TBL (Tampa bay Lightning) to TOR (Toronto Maple Leafs).  The row where Tm (Team) is TOT (Total)
### is his total stats.
### Handled this with the code below which identifiws traded players and sets M to 1 for all
### but the first row for that player. It also resets TOT to the last team the player played for.
### The code below worked OK until I tried it on the 2018 skater data and it failed. So rewrote it. 
### The code for this in my code module looks different from this.

In [5]:
l = len(df) - 2
i = 0

while i <= l:
    if df.iat[i,3] == "TOT":
        n = df.iat[i,1]
        pos = i
        temp_team = ""
        i += 1
        while df.iat[i,1] == n:
            df.iat[i,31] = 1
            temp_team = df.iat[i,3]
            i += 1
        df.iat[pos,3] = temp_team
        
    else:
        i += 1

In [ ]:
### Now remove the rows where M = 1

In [6]:
is_not_dup = df['M'] == 0
df0 =  df[is_not_dup]

In [23]:
### A player's name is followed by a \ and a code. Need to remove that.

In [7]:
for i in range(len(df0)):
    temp = df0.iat[i,1]
    place = temp.find('\\')
    df0.iat[i,1] = temp[0:place]

In [9]:
df0.iloc[20:25,:]

,Rk,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,...,HIT,FOW,FOL,FO%,adjPTS,adj+/-,adjTOI,M,PEval,Status
20,21,Keith Aulie,22,TOR,D,36,0,3,3,-7,...,78,0.0,0.0,NaN,6.833333,-15.944444,1102.444444,0,0.0,0
23,22,Sean Avery,31,NYR,C,15,3,0,3,2,...,10,0.0,3.0,0.0,16.400000,10.933333,574.000000,0,0.0,0
24,23,Anton Babchuk,27,CGY,D,32,2,8,10,2,...,17,0.0,0.0,NaN,25.625000,5.125000,1189.000000,0,0.0,0
25,24,David Backes,27,STL,RW,82,24,30,54,15,...,226,658.0,695.0,48.6,54.000000,15.000000,1640.000000,0,0.0,0
26,25,Mikael Backlund,22,CGY,C,41,4,7,11,-13,...,25,225.0,271.0,45.4,22.000000,-26.000000,1262.000000,0,0.0,0


In [ ]:
### The data for Keith Aulie has been reduced to one row now, and the code that followed player's
### names has been removed.

In [75]:
### Remove first and second year players - automatically exempt from expansion draft
### This operation is done later in my code module than it is here. I decided it was better to 
### perform calculations (player evaluation, team evaluation, etc() before removing these players.

In [10]:
df_deb_2010 = pd.read_csv("debuts2010.csv",header=0, encoding='utf_16')

In [11]:
df_deb_2011 = pd.read_csv("debuts2011.csv",header=0, encoding='utf_16')

In [12]:
debs2010 = list(df_deb_2010['Name'])

In [13]:
debs2011 = list(df_deb_2011['Name'])

In [14]:
for i in range(len(df0)):
    if df0.iat[i,1] in debs2010 or df0.iat[i,1] in debs2011:
        df0.iat[i,33] = 1

In [16]:
df1 = df0[df0['Status'] == 0]

In [16]:
### Reduce the variable list to only those variables needed.
### Also done later in my module than here.

In [17]:
df2 = df1.iloc[:,[1,2,3,4,5,9,11,28,29,30,32]]

In [18]:
df2.head(1)

,Player,Age,Tm,Pos,GP,+/-,PS,adjPTS,adj+/-,adjTOI,PEval
0,Justin Abdelkader,24,DET,LW,81,4,1.6,22.271605,4.049383,1009.308642,0.0


In [9]:
### Now need base stats for wings, centers, and defensemen.
### Probablky have more stats than I need here.

In [10]:
### Centers

In [19]:
df2_centers = df2[df2['Pos'] == 'C']

In [20]:
avg_Points_centers = df2_centers['adjPTS'].mean()
avg_PlusMinus_centers = df2_centers['adj+/-'].mean()
avg_TimeOnIce_centers = df2_centers['adjTOI'].mean()
std_Points_centers = df2_centers['adjPTS'].std()
std_PlusMinus_centers = df2_centers['adj+/-'].std()
std_TimeOnInce_centers = df2_centers['adjTOI'].std()

In [121]:
### Defensemen

In [21]:
df2_defense = df2[df2['Pos'] == 'D']

In [22]:
avg_Points_defense = df2_defense['adjPTS'].mean()
avg_PlusMinus_defense = df2_defense['adj+/-'].mean()
avg_TimeOnIce_defense = df2_defense['adjTOI'].mean()
std_Points_defense = df2_defense['adjPTS'].std()
std_PlusMinus_defense = df2_defense['adj+/-'].std()
std_TimeOnInce_defense = df2_defense['adjTOI'].std()

In [23]:
### Wings

In [23]:
df2_wings = df2[(df2['Pos'] == 'RW') | (df2['Pos'] == 'LW')]

In [24]:
avg_Points_wings = df2_wings['adjPTS'].mean()
avg_PlusMinus_wings = df2_wings['adj+/-'].mean()
avg_TimeOnIce_wings = df2_wings['adjTOI'].mean()
std_Points_wings = df2_wings['adjPTS'].std()
std_PlusMinus_wings = df2_wings['adj+/-'].std()
std_TimeOnInce_wings = df2_wings['adjTOI'].std()

In [ ]:
### Need a list of teams.

In [25]:
teams = df2['Tm'].unique()


In [26]:
teams

array(['DET', 'PIT', 'VAN', 'OTT', 'CAR', 'MIN', 'WSH', 'NYR', 'WPG',
       'TOR', 'STL', 'PHX', 'CGY', 'NYI', 'DAL', 'EDM', 'CBJ', 'ANA',
       'FLA', 'TBL', 'BOS', 'NSH', 'NJD', 'CHI', 'MTL', 'BUF', 'SJS',
       'PHI', 'LAK', 'COL'], dtype=object)

In [ ]:
### Calculate some team stats

In [27]:
team_plus_minus = []
for team in teams:
    df2_myteam = df2[df2['Tm'] == team]
    team_plus_minus.append(df2_myteam['+/-'].sum())


In [28]:
team_Avg_PlusMinus = 0
tot = 0
for i in team_plus_minus:
    tot += i
team_Avg_PlusMinus = tot/len(team_plus_minus)

    
    

In [29]:
import math
team_Std_PlusMinus = 0
tot = 0
for i in team_plus_minus:
    tot += (i - team_Avg_PlusMinus)**2
team_Std_PlusMinus = math.sqrt(tot/len(team_plus_minus))   
   

In [133]:
### Get data for a team

In [31]:
df2.head(1)

,Player,Age,Tm,Pos,GP,+/-,PS,adjPTS,adj+/-,adjTOI,PEval
0,Justin Abdelkader,24,DET,LW,81,4,1.6,22.271605,4.049383,1009.308642,0.0


In [ ]:
### The code below gets the data for each team, and then calculates the PEval for 
### each player on the team.  It decides which scheme to use to protect players 
### then marks the players to be protected. Then it removes those players. 
### In my code module there are more methods to choose from for protecting players.
### Also, in my code module the player evaluation is done differently.

In [32]:
path = './2011'
temp_df = None
for team in teams:
    fname = 'unprotected2011.csv'
    fname2 = 'protected' + team  + '.csv'
    
    df2_myteam = df2[df2['Tm'] == team]   
    
    for i in range(len(df2_myteam)):        
        if df2_myteam.iat[i,3] == 'D':
            player_z = (df2_myteam.iat[i,8] - avg_PlusMinus_defense)/std_PlusMinus_defense            
            
        elif df2_myteam.iat[i,3] == 'C':
            player_z = (df2_myteam.iat[i,8] - avg_PlusMinus_centers)/std_PlusMinus_centers           
           
        else:
            player_z = (df2_myteam.iat[i,8] - avg_PlusMinus_wings)/std_PlusMinus_wings            
            
        df2_myteam.iat[i,10] = player_z      
                
    df2_myteam.sort_values(by=['adjTOI'],axis=0,inplace=True, ascending=False)   
    # Played small number of games indicating a call up near the end of the season
    df2_red_gp = df2_myteam[df2_myteam['GP'] > 41]    
    # Too old - no point in protecting these players.
    df2_red_age2 = df2_red_gp[df2_red_gp['Age'] < 39]
    # Add a column to indicate which players are being protected.
    df2_red_age2['Protect'] = 0
    
    # Determine whether to protect any eight skaters or 7 fwd and 3 def.   
    num_def_protected = 0
    def_tot_count = 0        
    for i in range(len(df2_red_age2)):
        if df2_red_age2.iat[i,3] == 'D':            
            def_tot_count += 1
            
    def_top8_count = 0
    for i in range(8):
        if df2_red_age2.iat[i,3] == 'D':            
            def_top8_count += 1
            
    if def_top8_count <= 3:
        num_def_protected = 3       
    if def_top8_count > 4:
        num_def_protected = 4
    if def_top8_count == 4:
        if def_tot_count > 4:
            num_def_protected = 4
        else:
            num_def_protected = 3
    
                
    def_count = 0   
    
    for i in range(len(df2_red_age2)):
        if df2_red_age2.iat[i,3] == 'D':
            df2_red_age2.iat[i,11] = 1
            def_count += 1
        if def_count == num_def_protected:
            break
            
    num_fwd_protected = 0        
    if num_def_protected == 3:
        num_fwd_protected = 7
    else:
        num_fwd_protected = 4
        
    fwd_count = 0    
    for i in range(len(df2_red_age2)):
        if not df2_red_age2.iat[i,3] == 'D':
            df2_red_age2.iat[i,11] = 1
            fwd_count += 1
        if fwd_count == num_fwd_protected:
            break 
    
    dfx = df2_red_age2[df2_red_age2['Protect']==1]
    dfx.to_csv(os.path.join(path,fname2), index = False)
    df3_myteam = df2_red_age2[df2_red_age2['Protect'] == 0]
    if temp_df is None:
        temp_df = df3_myteam
    else:
        temp_df = pd.concat([temp_df,df3_myteam])    
    
temp_df.to_csv(os.path.join(path,fname), index=False)        
        

/home/newton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/newton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/newton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/newton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.

/home/newton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/newton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/newton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/newton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be s

In [ ]:
### Read the resulting data - unprotected player list.

In [33]:
df_roster = pd.read_csv('./2011/unprotected2011.csv')

In [ ]:
### Now sort the by minutes played. In my code module the data is sorted by player score.

In [34]:
eg = df_roster.sort_values(by=['adjTOI'],ascending=False, axis=0)

In [35]:
eg.head(5)

,Player,Age,Tm,Pos,GP,+/-,PS,adjPTS,adj+/-,adjTOI,PEval,Protect
214,Pavel Kubina,34,PHI,D,69,-2,2.3,17.826087,-2.376812,1580.579710,-0.077866,0
84,Derek Morris,33,PHX,D,59,-12,1.4,15.288136,-16.677966,1556.610169,-0.821317,0
30,Jared Cowen,21,OTT,D,82,-4,2.7,17.000000,-4.000000,1549.000000,-0.162248,0
78,Roman Polak,25,STL,D,77,6,3.7,11.714286,6.389610,1547.350649,0.377860,0
160,Joe Corvo,34,BOS,D,75,10,4.7,27.333333,10.933333,1542.693333,0.614067,0


In [ ]:
### The code below selects players. Reads from the top of the player list to the bottom.
### Selects the best players, one from each team. 
### There are other ways to do this, and in my code module more options are available for
### the user.

In [36]:
num_fwds = 0
num_lws = 0
num_rws = 0
num_defs = 0
num_ctrs = 0
team_tracker = []
max_fwds = 14
max_defs = 9
min_lws = 3
min_rws = 3
min_ctrs = 4
max_players = 23
num_players = 0
min_lws_reached = False
min_rws_reached = False
min_ctrs_reached = False

for i in range(len(eg)):
    tm = eg.iat[i,2]
    if not tm in team_tracker and num_players < max_players:
        pos = eg.iat[i,3]
        if not pos == 'D' and num_fwds < max_fwds:
            if pos == 'LW':
                if min_lws_reached == True:
                    players_needed = 0
                    if min_rws_reached == False:
                        players_needed += (min_rws - num_rws)
                    if min_ctrs_reached == False:
                        players_needed += (min_ctrs - num_ctrs)
                    num_fwds_needed = max_fwds - num_fwds
                    if players_needed < num_fwds_needed:
                        #addit
                        num_players += 1
                        num_lws += 1
                        num_fwds += 1
                        eg.iat[i,11] = 1
                        team_tracker.append(tm)
                        if num_lws >= min_lws:
                            min_lws_reached = True
               
                else:
                    # addit
                    num_players += 1
                    num_lws += 1
                    num_fwds += 1
                    eg.iat[i,11] = 1
                    team_tracker.append(tm)
                    if num_lws >= min_lws:
                        min_lws_reached = True
               
            if pos == 'RW':
                if min_rws_reached == True:
                    players_needed = 0
                    if min_lws_reached == False:
                        players_needed += (min_lws - num_lws)
                    if min_ctrs_reached == False:
                        players_needed += (min_ctrs - num_ctrs)
                    num_fwds_needed = max_fwds - num_fwds
                    if players_needed < num_fwds_needed:
                        #addit
                        num_players += 1
                        num_rws += 1
                        num_fwds += 1
                        eg.iat[i,11] = 1
                        team_tracker.append(tm)
                        if num_rws >= min_rws:
                            min_rws_reached = True
               
                else:
                    # addit
                    num_players += 1
                    num_rws += 1
                    num_fwds += 1
                    eg.iat[i,11] = 1
                    team_tracker.append(tm)
                    if num_rws >= min_rws:
                        min_rws_reached = True
                        
            if pos == 'C':
                if min_ctrs_reached == True:
                    players_needed = 0
                    if min_rws_reached == False:
                        players_needed += (min_rws - num_rws)
                    if min_lws_reached == False:
                        players_needed += (min_lws - num_lws)
                    num_fwds_needed = max_fwds - num_fwds
                    if players_needed < num_fwds_needed:
                        #addit
                        num_players += 1
                        num_ctrs += 1
                        num_fwds += 1
                        eg.iat[i,11] = 1
                        team_tracker.append(tm)
                        if num_ctrs >= min_ctrs:
                            min_ctrs_reached = True
               
                else:
                    # addit
                    num_players += 1
                    num_ctrs += 1
                    num_fwds += 1
                    eg.iat[i,11] = 1
                    team_tracker.append(tm)
                    if num_ctrs >= min_ctrs:
                        min_ctrs_reached = True
                        
            
        else:    
            if num_defs < max_defs:
                # add it
                team_tracker.append(tm)
                eg.iat[i,11] = 1
                num_defs += 1
                num_players += 1
    

In [ ]:
### Selected players are marked with a 1.  

In [37]:
eg1 = eg[eg['Protect']==1]

In [58]:
eg1

,Player,Age,Tm,Pos,GP,+/-,PS,adjPTS,adj+/-,adjTOI,PEval,Protect
214,Pavel Kubina,34,PHI,D,69,-2,2.3,17.826087,-2.376812,1580.579710,-0.077866,1
84,Derek Morris,33,PHX,D,59,-12,1.4,15.288136,-16.677966,1556.610169,-0.821317,1
30,Jared Cowen,21,OTT,D,82,-4,2.7,17.000000,-4.000000,1549.000000,-0.162248,1
78,Roman Polak,25,STL,D,77,6,3.7,11.714286,6.389610,1547.350649,0.377860,1
160,Joe Corvo,34,BOS,D,75,10,4.7,27.333333,10.933333,1542.693333,0.614067,1
227,Ryan Wilson,24,COL,D,59,11,4.8,29.186441,15.288136,1535.762712,0.840454,1
196,Robyn Regehr,31,BUF,D,76,-12,1.1,5.394737,-12.947368,1527.789474,-0.627380,1
95,Curtis Glencross,29,CGY,LW,67,-13,6.0,58.746269,-15.910448,1478.447761,-0.679264,1
180,Anton Volchenkov,29,NJD,D,72,3,3.4,12.527778,3.416667,1473.722222,0.223311,1
113,Philip Larsen,22,DAL,D,55,11,3.3,16.400000,16.400000,1473.018182,0.898254,1


In [ ]:
### Calculate the team score.

In [38]:
exp_team_tot = eg1['adj+/-'].sum()

In [39]:
team_score = (exp_team_tot - team_Avg_PlusMinus) / team_Std_PlusMinus

In [ ]:
### Team score is the number of standard deviations above the mean
### based on the team stats calculated above.

In [40]:
team_score

0.21650636890514385

In [ ]:
### So this would be slightly better than the average team in the league.  